### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import copy
import random
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# 平均値と標準偏差を計算するためのCIFAR100データセットの前処理を定義
pre_transform_cifar100 = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのCIFAR100データセットの読み込み
pre_train_dataset_cifar100 = datasets.CIFAR100(root='/content/data/', download=True, transform=pre_transform_cifar100)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_cifar100 = 0.0
pre_std_cifar100 = 0.0
pre_total_samples_cifar100 = len(pre_train_dataset_cifar100)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_cifar100:
    pre_image, _ = data
    pre_mean_cifar100 += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_cifar100 += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_cifar100 /= pre_total_samples_cifar100
pre_std_cifar100 /= pre_total_samples_cifar100

print("データセット全体の平均値: ", pre_mean_cifar100)
print("データセット全体の標準偏差: ", pre_std_cifar100)

100%|██████████| 169001437/169001437 [00:12<00:00, 13223134.42it/s]


Extracting /content/data/cifar-100-python.tar.gz to /content/data/
データセット全体の平均値:  tensor([0.5071, 0.4866, 0.4409])
データセット全体の標準偏差:  tensor([0.2009, 0.1984, 0.2023])


In [5]:
# 学習用のCIFAR100データセットの前処理を定義
train_transform_cifar100 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar100, std=pre_std_cifar100)
    ])
# テスト用のCIFAR100データセットの前処理を定義
test_transform_cifar100 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar100, std=pre_std_cifar100)
    ])

# 学習用のCIFAR100データセットの読み込み
train_dataset_cifar100 = torchvision.datasets.CIFAR100(root='/content/data/', train=True, transform=train_transform_cifar100, download=True)
# テスト用のCIFAR100データセットの読み込み
test_dataset_cifar100 = torchvision.datasets.CIFAR100(root='/content/data/', train=False, transform=test_transform_cifar100, download=True)

# 学習用のCIFAR100データローダーを作成
train_loader_cifar100 = torch.utils.data.DataLoader(dataset=train_dataset_cifar100, batch_size=512, shuffle=True, num_workers=2)
# テスト用のCIFAR100データローダーを作成
test_loader_cifar100 = torch.utils.data.DataLoader(dataset=test_dataset_cifar100, batch_size=512, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## モデルの実装

In [6]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [7]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [8]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [9]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [10]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBuildingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.bn1 = NonAffineBatchNorm(in_channels)
        self.conv1 = supermaskconv3x3(in_channels, out_channels, stride)
        self.bn2 = NonAffineBatchNorm(out_channels)
        self.conv2 = supermaskconv3x3(out_channels, out_channels)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != out_channels or stride > 1:
            self.shortcut = supermaskconv1x1(in_channels, out_channels, stride)
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        # 残差写像と恒等写像の要素毎の和を計算
        out += self.shortcut(x)
        return out

In [11]:
class SupermaskWideResNet(nn.Module):
    def __init__(self, block, depth, k, num_classes=10):
        super().__init__()
        assert (depth - 4) % 6 == 0, "depth should be 6n + 4"
        n = (depth - 4) // 6
        channels = [16, 16 * k, 32 * k, 64 * k]
        self.conv1 = supermaskconv3x3(3, channels[0])
        # Residual Blocks（1)
        self.layer1 = self._make_layer(block, channels[0], channels[1], n)
        # Residual Blocks（2）
        self.layer2 = self._make_layer(block, channels[1], channels[2], n, stride=2)
        # Residual Blocks（3）
        self.layer3 = self._make_layer(block, channels[2], channels[3], n, stride=2)
        self.bn = NonAffineBatchNorm(channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channels[3], num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        layers = []
        # 最初の Residual Block（stride=stride）
        layers.append(block(in_channels, out_channels, stride))
        # 残りの Residual Block（stride=1）
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [13]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.999_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 1.0 %, Loss: 4.6058
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 1.0 %, Loss: 4.6061
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 0.822 %, Loss: 4.6046
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 0.858 %, Loss: 4.6055
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 0.912 %, Loss: 4.6068
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 0.864 %, Loss: 4.6080
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 0.936 %, Loss: 4.6058
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 0.898 %, Loss: 4.6073
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 0.908 %, Loss: 4.6086
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 0.872 %, Loss: 4.6072
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 0.948 %, Loss: 4.6060
Epoch [11/50

In [14]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [15]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [16]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [17]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 1.0 %


In [18]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [19]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.995_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 4.88 %, Loss: 4.3367
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 13.048 %, Loss: 4.1033
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 22.474 %, Loss: 3.0170
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 27.984 %, Loss: 2.8080
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.186 %, Loss: 2.7679
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 31.792 %, Loss: 2.7001
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 32.16 %, Loss: 2.9023
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 33.408 %, Loss: 2.6110
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 34.866 %, Loss: 2.5699
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 35.618 %, Loss: 2.4940
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 36.124 %, Loss: 2.4166


In [20]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [21]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [22]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [23]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 44.34 %


In [24]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [25]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.99_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 4.71 %, Loss: 4.2969
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 14.072 %, Loss: 4.0258
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 24.656 %, Loss: 2.9359
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 30.536 %, Loss: 2.8412
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 33.256 %, Loss: 2.5289
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 35.192 %, Loss: 2.4012
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.98 %, Loss: 2.4537
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.252 %, Loss: 2.4564
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 38.648 %, Loss: 2.3390
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 39.408 %, Loss: 2.3951
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 40.412 %, Loss: 2.3491


In [26]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [27]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [28]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [29]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 50.15 %


In [30]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [31]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.95_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.716 %, Loss: 4.1947
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 13.586 %, Loss: 3.8826
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 21.714 %, Loss: 3.1225
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 28.202 %, Loss: 2.6982
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 31.826 %, Loss: 2.5374
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 34.278 %, Loss: 2.5072
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.402 %, Loss: 2.4011
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.94 %, Loss: 2.5374
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 38.912 %, Loss: 2.2793
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.41 %, Loss: 2.3602
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 41.348 %, Loss: 2.1807


In [32]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [33]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [34]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [35]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.44 %


In [36]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [37]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.9_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 6.022 %, Loss: 4.1644
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 13.574 %, Loss: 3.8762
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 20.286 %, Loss: 3.0265
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 26.794 %, Loss: 2.8266
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.656 %, Loss: 2.5475
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.306 %, Loss: 2.7853
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 34.756 %, Loss: 2.6046
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.22 %, Loss: 2.3489
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 38.604 %, Loss: 2.3818
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.048 %, Loss: 2.3804
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 40.858 %, Loss: 2.3889

In [38]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [39]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [40]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [41]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.06 %


In [42]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [43]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.7_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.676 %, Loss: 4.2255
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 12.002 %, Loss: 4.0286
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 19.606 %, Loss: 3.1067
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 26.212 %, Loss: 2.9099
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.392 %, Loss: 2.6654
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.562 %, Loss: 2.5126
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.124 %, Loss: 2.5860
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.328 %, Loss: 2.5986
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.4 %, Loss: 2.2659
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.364 %, Loss: 2.2413
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 41.082 %, Loss: 2.3270


In [44]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [45]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [46]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [47]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.01 %


In [48]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [49]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.5_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.106 %, Loss: 4.2302
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 12.276 %, Loss: 3.9433
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 19.42 %, Loss: 3.1438
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 25.782 %, Loss: 2.9600
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.476 %, Loss: 2.6506
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.416 %, Loss: 2.4220
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.282 %, Loss: 2.5359
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.392 %, Loss: 2.2772
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.178 %, Loss: 2.4224
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.338 %, Loss: 2.2789
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 41.282 %, Loss: 2.2815

In [50]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [51]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [52]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [53]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.66 %


In [54]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [55]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.999_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 2.908 %, Loss: 4.4327
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 8.306 %, Loss: 4.2829
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 14.098 %, Loss: 3.4848
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 17.962 %, Loss: 3.3479
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 19.534 %, Loss: 3.1932
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 20.34 %, Loss: 3.1484
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 20.428 %, Loss: 3.2670
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 21.344 %, Loss: 3.2714
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 21.962 %, Loss: 3.2500
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 22.638 %, Loss: 3.2256
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 23.082 %, Loss: 3.2274


In [56]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [57]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [58]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [59]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 27.39 %


In [60]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [61]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.995_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 4.966 %, Loss: 4.3411
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 12.666 %, Loss: 4.1011
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 21.778 %, Loss: 2.9889
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 26.516 %, Loss: 2.8600
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 28.818 %, Loss: 2.7075
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 29.694 %, Loss: 2.7622
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 30.684 %, Loss: 2.7747
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 32.12 %, Loss: 2.5854
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 33.082 %, Loss: 2.6937
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 33.518 %, Loss: 2.7289
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 34.06 %, Loss: 2.6667


In [62]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [63]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [64]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [65]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 42.03 %


In [66]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [67]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.99_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.558 %, Loss: 4.2711
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 14.226 %, Loss: 4.0231
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 25.276 %, Loss: 2.9027
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 31.63 %, Loss: 2.6971
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 34.126 %, Loss: 2.6127
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 35.63 %, Loss: 2.5276
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 36.614 %, Loss: 2.5193
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 38.064 %, Loss: 2.3230
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.43 %, Loss: 2.3558
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.482 %, Loss: 2.2271
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 40.792 %, Loss: 2.2665
E

In [68]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [69]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [70]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [71]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.99 %


In [72]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [73]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.95_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.732 %, Loss: 4.2158
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 14.024 %, Loss: 3.9188
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 22.924 %, Loss: 3.0425
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 29.06 %, Loss: 2.6123
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 33.048 %, Loss: 2.4996
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 35.41 %, Loss: 2.4582
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 37.06 %, Loss: 2.4245
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 38.738 %, Loss: 2.4288
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.968 %, Loss: 2.3988
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 41.312 %, Loss: 2.2333
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 42.074 %, Loss: 2.2325
E

In [74]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [75]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [76]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [77]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.35 %


In [78]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [79]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.9_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.398 %, Loss: 4.2004
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 12.844 %, Loss: 3.8878
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 20.478 %, Loss: 3.1279
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 27.034 %, Loss: 2.7914
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 31.296 %, Loss: 2.5613
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.486 %, Loss: 2.4542
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 34.948 %, Loss: 2.4450
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.57 %, Loss: 2.4311
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 38.826 %, Loss: 2.3436
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.036 %, Loss: 2.1374
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 40.802 %, Loss: 2.2089

In [80]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [81]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [82]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [83]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 50.72 %


In [84]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [85]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.7_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 6.28 %, Loss: 4.1835
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 12.816 %, Loss: 3.9315
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 20.21 %, Loss: 3.0548
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 26.212 %, Loss: 2.8830
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.282 %, Loss: 2.5688
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 32.834 %, Loss: 2.4426
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 34.968 %, Loss: 2.4532
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.438 %, Loss: 2.3475
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 38.986 %, Loss: 2.4530
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 39.604 %, Loss: 2.5355
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 40.868 %, Loss: 2.1762


In [86]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [87]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [88]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [89]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.12 %


In [90]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [91]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.5_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.4 %, Loss: 4.2357
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 11.62 %, Loss: 3.9726
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 19.454 %, Loss: 3.0973
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 26.306 %, Loss: 2.9551
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.668 %, Loss: 2.7534
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.276 %, Loss: 2.6828
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.848 %, Loss: 2.4809
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.77 %, Loss: 2.3202
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.152 %, Loss: 2.3700
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.268 %, Loss: 2.3141
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 41.286 %, Loss: 2.2520
Ep

In [92]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [93]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [94]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [95]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.19 %


In [96]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [97]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.999_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 1.0 %, Loss: 4.6039
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 0.918 %, Loss: 4.6030
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 0.912 %, Loss: 4.6054
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 0.894 %, Loss: 4.6068
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 0.926 %, Loss: 4.6072
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 0.896 %, Loss: 4.6059
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 0.908 %, Loss: 4.6074
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 0.906 %, Loss: 4.6072
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 0.936 %, Loss: 4.6073
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 0.928 %, Loss: 4.6074
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 0.864 %, Loss: 4.6061
Epoch [11/

In [98]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [99]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [100]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [101]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 1.0 %


In [102]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [103]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.995_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.15 %, Loss: 4.3160
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 14.19 %, Loss: 4.0656
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 24.436 %, Loss: 2.9954
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 30.494 %, Loss: 2.7149
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 32.454 %, Loss: 2.7304
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.412 %, Loss: 2.5646
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 34.764 %, Loss: 2.6155
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 36.386 %, Loss: 2.3479
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 37.262 %, Loss: 2.4129
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 37.964 %, Loss: 2.4146
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 38.694 %, Loss: 2.5440


In [104]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [105]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [106]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [107]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 47.02 %


In [108]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [109]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.99_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 6.054 %, Loss: 4.2425
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 15.212 %, Loss: 4.0058
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 25.628 %, Loss: 2.7707
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 32.54 %, Loss: 2.6312
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 35.386 %, Loss: 2.5349
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 37.092 %, Loss: 2.4273
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 38.348 %, Loss: 2.2774
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 39.552 %, Loss: 2.4171
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 41.34 %, Loss: 2.1688
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 42.168 %, Loss: 2.2473
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 42.84 %, Loss: 1.9631
E

In [110]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [111]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [112]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [113]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.9 %


In [114]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [115]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.95_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 6.376 %, Loss: 4.1538
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 14.752 %, Loss: 3.8632
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 21.598 %, Loss: 2.9916
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 28.198 %, Loss: 2.6581
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 31.956 %, Loss: 2.7905
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 34.438 %, Loss: 2.5461
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.802 %, Loss: 2.3165
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 38.224 %, Loss: 2.2962
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.344 %, Loss: 2.5288
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.984 %, Loss: 2.2553
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 41.638 %, Loss: 2.233

In [116]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [117]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [118]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [119]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.01 %


In [120]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [121]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.9_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 6.238 %, Loss: 4.1350
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 13.412 %, Loss: 3.8980
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 20.398 %, Loss: 3.0825
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 26.634 %, Loss: 2.9257
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.408 %, Loss: 2.6226
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.078 %, Loss: 2.4427
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 34.956 %, Loss: 2.5101
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.008 %, Loss: 2.2646
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 38.882 %, Loss: 2.3551
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 39.566 %, Loss: 2.3479
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 40.862 %, Loss: 2.419

In [122]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [123]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [124]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [125]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.03 %


In [126]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [127]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.7_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 4.946 %, Loss: 4.1929
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 12.538 %, Loss: 3.9462
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 19.998 %, Loss: 3.1612
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 25.962 %, Loss: 2.9487
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.506 %, Loss: 2.6849
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.738 %, Loss: 2.5751
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.436 %, Loss: 2.5313
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.412 %, Loss: 2.2094
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 38.872 %, Loss: 2.2825
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.61 %, Loss: 2.1959
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 40.886 %, Loss: 2.1982

In [128]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [129]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [130]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [131]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.36 %


In [132]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [133]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 100)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_from_CIFAR10_100epochs_pr0.5_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 5.762 %, Loss: 4.2113
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 12.712 %, Loss: 3.9149
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 19.966 %, Loss: 3.1003
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 25.986 %, Loss: 2.7814
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.126 %, Loss: 2.6914
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 33.524 %, Loss: 2.6593
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.662 %, Loss: 2.4761
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.572 %, Loss: 2.2825
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.582 %, Loss: 2.2700
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.594 %, Loss: 2.1829
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 41.588 %, Loss: 2.207

In [134]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [135]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [136]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [137]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.95 %


In [138]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()